# LangChain Expression Language (LCEL)

In [18]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [19]:
#!pip install pydantic==1.10.8

In [20]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [21]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [22]:
chain = prompt | model | output_parser

In [24]:
chain.invoke({"topic": "Engineering college"})

'Why did the engineering student bring a pencil to their exam? \n\nIn case they needed to draw a diagram of how they were going to fail.'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [25]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [26]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["Tharun lives at Poonamallee", "SEC stands for Saveetha Engineering College"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [27]:
retriever.get_relevant_documents("where does Tharun live?")

[Document(page_content='Tharun lives at Poonamallee'),
 Document(page_content='SEC stands for Saveetha Engineering College')]

In [28]:
retriever.get_relevant_documents("what does SEC stand for?")

[Document(page_content='SEC stands for Saveetha Engineering College'),
 Document(page_content='Tharun lives at Poonamallee')]

In [29]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [30]:
from langchain.schema.runnable import RunnableMap

In [31]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [36]:
chain.invoke({"question": "where does Tharun live?"})

'Tharun lives at Poonamallee.'

In [33]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [34]:
inputs.invoke({"question": "what does SEC stand for?"})

{'context': [Document(page_content='SEC stands for Saveetha Engineering College'),
  Document(page_content='Tharun lives at Poonamallee')],
 'question': 'what does SEC stand for?'}